In [21]:
required_packages <- c(
"caret",
"MASS",
"randomForest",
"e1071",
"ggplot2",
"dplyr",
"corrplot",
"nnet",
"naivebayes"
)

# Function to install and load packages
install_and_load_packages <- function(packages) {
  cat("Checking and installing required packages...\n")
  
  for (package in packages) {
    if (!require(package, character.only = TRUE, quietly = TRUE)) {
      cat(sprintf("Installing package: %s\n", package))
      install.packages(package, dependencies = TRUE)
      if (!require(package, character.only = TRUE, quietly = TRUE)) {
        stop(sprintf("Package '%s' installation failed", package))
      }
    } else {
      cat(sprintf("Package '%s' is already installed and loaded\n", package))
    }
  }
  cat("\nAll required packages are installed and loaded!\n\n")
}

# Install and load all required packages
install_and_load_packages(required_packages)


Checking and installing required packages...
Package 'caret' is already installed and loaded
Package 'MASS' is already installed and loaded
Package 'randomForest' is already installed and loaded
Package 'e1071' is already installed and loaded
Package 'ggplot2' is already installed and loaded
Package 'dplyr' is already installed and loaded
Package 'corrplot' is already installed and loaded
Package 'nnet' is already installed and loaded
Package 'naivebayes' is already installed and loaded

All required packages are installed and loaded!



In [22]:
library(caret)
library(MASS)
library(randomForest)
library(naivebayes)
library(e1071)
library(ggplot2)
library(dplyr)
library(corrplot)
library(nnet)

In [23]:
# Read the data
data <- read.table("a24_clas_app.txt", header = TRUE, sep=" ")

# Convert y to a factor
data$y <- as.factor(data$y)

# Make a copy of the original data for processing
data_cleaned <- data

# Identify excluded variables (0-13 values)
excluded_vars <- c("X46", "X47", "X48", "X49", "X50") 
numeric_vars <- names(data_cleaned)[sapply(data_cleaned, is.numeric) & !(names(data_cleaned) %in% excluded_vars) & names(data_cleaned) != "y"]

# Outlier Removal
# ------------------
# Calculate skewness and IQR filtering for outliers on selected numeric variables only
for (var in numeric_vars) {
  Q1 <- quantile(data_cleaned[[var]], 0.25, na.rm = TRUE)
  Q3 <- quantile(data_cleaned[[var]], 0.75, na.rm = TRUE)
  IQR_val <- Q3 - Q1
  
  lower_bound <- Q1 - 1.5 * IQR_val
  upper_bound <- Q3 + 1.5 * IQR_val
  
  # Remove rows with outliers in any numeric variable
  data_cleaned <- data_cleaned[!(data_cleaned[[var]] < lower_bound | data_cleaned[[var]] > upper_bound), ]
}

# Calculate the ratio of rows deleted
cat("Rows deleted due to outliers:", (1 - nrow(data_cleaned) / nrow(data)) * 100, "%\n")

# Scaling
# ----------
# Apply scaling to numeric variables only
preprocess_params <- preProcess(data_cleaned[, numeric_vars], method = c("center", "scale"))
data_scaled <- data_cleaned
data_scaled[, numeric_vars] <- predict(preprocess_params, data_cleaned[, numeric_vars])

# Separate datasets with and without excluded variables
data_with_excluded <- data_scaled
data_without_excluded <- data_scaled %>% select(-all_of(excluded_vars))

Rows deleted due to outliers: 27.6 %


In [24]:
# Define outer and inner folds
outer_folds <- 5
inner_folds <- 3

# Define classification-only methods
classification_methods <- c("multinom", "naive_bayes", "qda", "lda", "rpart", "rf")

# Function to evaluate classification model
evaluate_model <- function(model, test_data, true_labels) {
  predictions <- predict(model, newdata = test_data)
  
  # Ensure levels are specified in multiClassSummary
  levels <- levels(true_labels)
  metrics <- multiClassSummary(data.frame(pred = predictions, obs = true_labels), lev = levels)
  
  return(list(
    Accuracy = metrics$Accuracy,
    Sensitivity = metrics$Recall,
    Specificity = metrics$Specificity,
    F1 = metrics$F1
  ))
}

# Store results
results <- data.frame(Model = character(), Variable_Set = character(), Fold = integer(),
                      Accuracy = numeric(), Sensitivity = numeric(), Specificity = numeric(), F1 = numeric())

# Loop through each classification model and both datasets
for (method in classification_methods) {
  for (variable_set in c("with_excluded", "without_excluded")) {
    
    data_set <- if (variable_set == "with_excluded") data_with_excluded else data_without_excluded
    
    for (outer_fold in 1:outer_folds) {
      outer_train_index <- createFolds(data_set$y, k = outer_folds, list = TRUE, returnTrain = TRUE)[[outer_fold]]
      outer_train_data <- data_set[outer_train_index, ]
      outer_test_data <- data_set[-outer_train_index, ]
      
      best_inner_model <- NULL
      best_inner_accuracy <- 0
      
      for (inner_fold in 1:inner_folds) {
        inner_train_index <- createFolds(outer_train_data$y, k = inner_folds, list = TRUE, returnTrain = TRUE)[[inner_fold]]
        inner_train_data <- outer_train_data[inner_train_index, ]
        inner_val_data <- outer_train_data[-inner_train_index, ]
        
        # Train the model
        model <- train(y ~ ., data = inner_train_data, method = method, trControl = trainControl(method = "cv", number = inner_folds), metric = "Accuracy")
        
        # Get accuracy on validation set
        val_pred <- predict(model, newdata = inner_val_data)
        val_accuracy <- mean(val_pred == inner_val_data$y)
        
        # Update best model if necessary
        if (val_accuracy > best_inner_accuracy) {
          best_inner_model <- model
          best_inner_accuracy <- val_accuracy
        }
      }
      
      # Evaluate the best inner model on outer test set
      outer_metrics <- evaluate_model(best_inner_model, outer_test_data, outer_test_data$y)
      
      # Store results
      results <- rbind(results, data.frame(Model = method, Variable_Set = variable_set, Fold = outer_fold,
                                           Accuracy = outer_metrics$Accuracy,
                                           Sensitivity = outer_metrics$Sensitivity,
                                           Specificity = outer_metrics$Specificity,
                                           F1 = outer_metrics$F1))
    }
  }
}

# Print and Plot Final Results
cat("\nClassification Results:\n")
print(results)

# Plot results
ggplot(results, aes(x = Model, y = Accuracy, fill = Variable_Set)) +
  geom_boxplot() +
  labs(title = "Model Accuracy across Nested Cross-Validation Folds",
       x = "Model", y = "Accuracy")

ggplot(results, aes(x = Model, y = Sensitivity, fill = Variable_Set)) +
  geom_boxplot() +
  labs(title = "Model Sensitivity across Nested Cross-Validation Folds",
       x = "Model", y = "Sensitivity")

ggplot(results, aes(x = Model, y = Specificity, fill = Variable_Set)) +
  geom_boxplot() +
  labs(title = "Model Specificity across Nested Cross-Validation Folds",
       x = "Model", y = "Specificity")

ggplot(results, aes(x = Model, y = F1, fill = Variable_Set)) +
  geom_boxplot() +
  labs(title = "Model F1 across Nested Cross-Validation Folds",
       x = "Model", y = "F1")


# weights:  156 (102 variable)
initial  value 140.622373 
iter  10 value 69.332750
iter  20 value 37.702710
iter  30 value 20.341340
iter  40 value 3.166699
iter  50 value 0.013181
final  value 0.000074 
converged
# weights:  156 (102 variable)
initial  value 140.622373 
iter  10 value 71.619008
iter  20 value 47.665652
iter  30 value 42.906933
iter  40 value 41.328498
iter  50 value 41.131396
iter  60 value 41.109130
iter  70 value 41.108523
final  value 41.108518 
converged
# weights:  156 (102 variable)
initial  value 140.622373 
iter  10 value 69.335180
iter  20 value 37.718825
iter  30 value 20.435934
iter  40 value 4.055007
iter  50 value 1.108322
iter  60 value 1.031528
iter  70 value 0.948186
iter  80 value 0.886681
iter  90 value 0.861517
iter 100 value 0.843646
final  value 0.843646 
stopped after 100 iterations
# weights:  156 (102 variable)
initial  value 142.819598 
iter  10 value 57.207446
iter  20 value 30.370845
iter  30 value 9.329850
iter  40 value 0.320099
iter  50 v

ERROR: Error in metrics$Accuracy: $ operator is invalid for atomic vectors


In [4]:
# # Load required libraries
# library(tidyverse)
# library(ggplot2)
# library(caret)
# library(MASS)
# library(rpart)
# library(randomForest)
# library(corrplot)
# library(e1071)
# library(pROC)
# library(moments)
# library(gridExtra)

# # ---- 1. Data Loading and Initial Setup ----
# data <- read.table("a24_clas_app.txt", header = TRUE,sep = " ")
# X <- data[, 1:50]  # Features
# y <- as.factor(data[, 51])  # Target variable

# # ---- 2. Feature Analysis Functions ----
# analyze_features <- function(X) {
#   feature_stats <- data.frame(
#     mean = colMeans(X),
#     sd = apply(X, 2, sd),
#     median = apply(X, 2, median),
#     skewness = apply(X, 2, skewness),
#     kurtosis = apply(X, 2, kurtosis),
#     zeros = colSums(X == 0),
#     unique_values = apply(X, 2, function(x) length(unique(x)))
#   )
#   return(feature_stats)
# }

# detect_outliers <- function(X, method = "iqr", threshold = 1.5) {
#   outliers_summary <- list()
#   for (col in colnames(X)) {
#     x <- X[[col]]
#     if (method == "iqr") {
#       Q1 <- quantile(x, 0.25)
#       Q3 <- quantile(x, 0.75)
#       IQR <- Q3 - Q1
#       lower_bound <- Q1 - threshold * IQR
#       upper_bound <- Q3 + threshold * IQR
#       outliers_summary[[col]] <- list(
#         n_outliers = sum(x < lower_bound | x > upper_bound),
#         lower_bound = lower_bound,
#         upper_bound = upper_bound
#       )
#     }
#   }
#   return(outliers_summary)
# }

# # ---- 3. Preprocessing Function ----
# preprocess_data <- function(X, scale = TRUE, remove_correlated = FALSE, 
#                           correlation_threshold = 0.8, handle_outliers = TRUE) {
#   # Handle outliers if requested
#   if (handle_outliers) {
#     outliers <- detect_outliers(X)
#     for (col in colnames(X)) {
#       lower_bound <- outliers[[col]]$lower_bound
#       upper_bound <- outliers[[col]]$upper_bound
#       X[[col]][X[[col]] < lower_bound] <- lower_bound
#       X[[col]][X[[col]] > upper_bound] <- upper_bound
#     }
#   }
  
#   # Remove highly correlated features if requested
#   if (remove_correlated) {
#     correlation_matrix <- cor(X)
#     highly_correlated <- findCorrelation(correlation_matrix, 
#                                        cutoff = correlation_threshold)
#     if (length(highly_correlated) > 0) {
#       X <- X[, -highly_correlated]
#     }
#   }
  
#   # Scale features if requested
#   if (scale) {
#     X <- scale(X)
#   }
  
#   return(X)
# }

# # ---- 4. Model Evaluation Functions ----
# calculate_metrics <- function(pred, actual) {
#   confusion_mat <- confusionMatrix(pred, actual)
#   metrics <- list(
#     accuracy = confusion_mat$overall["Accuracy"],
#     sensitivity = mean(confusion_mat$byClass[, "Sensitivity"]),
#     specificity = mean(confusion_mat$byClass[, "Specificity"])
#   )
#   return(metrics)
# }

# # ---- 5. Nested Cross-Validation Function ----
# nested_cv <- function(X, y, outer_folds = 5, inner_folds = 5, 
#                      model_type = c("logistic", "lda", "qda", "naive_bayes", "tree", "rf"),
#                      preprocess_params = list(scale = TRUE, 
#                                            remove_correlated = FALSE,
#                                            handle_outliers = TRUE)) {
#   outer_cv <- createFolds(y, k = outer_folds, list = TRUE)
#   results <- list()
#   predictions <- numeric(length(y))
  
#   for (fold in 1:outer_folds) {
#     # Split data
#     train_indices <- unlist(outer_cv[-fold])
#     test_indices <- outer_cv[[fold]]
    
#     X_train <- X[train_indices, ]
#     y_train <- y[train_indices]
#     X_test <- X[test_indices, ]
#     y_test <- y[test_indices]
    
#     # Preprocess data
#     X_train_processed <- preprocess_data(X_train, 
#                                        scale = preprocess_params$scale,
#                                        remove_correlated = preprocess_params$remove_correlated,
#                                        handle_outliers = preprocess_params$handle_outliers)
    
#     # Apply same preprocessing to test data
#     X_test_processed <- predict(preprocess(X_train_processed), X_test)
    
#     # Train model based on type
#     if (model_type == "logistic") {
#       model <- multinom(y_train ~ ., data = as.data.frame(X_train_processed))
#       pred <- predict(model, newdata = as.data.frame(X_test_processed), type = "class")
#     } else if (model_type == "lda") {
#       model <- lda(X_train_processed, y_train)
#       pred <- predict(model, X_test_processed)$class
#     } else if (model_type == "qda") {
#       model <- qda(X_train_processed, y_train)
#       pred <- predict(model, X_test_processed)$class
#     } else if (model_type == "naive_bayes") {
#       model <- naiveBayes(X_train_processed, y_train)
#       pred <- predict(model, X_test_processed)
#     } else if (model_type == "tree") {
#       model <- rpart(y_train ~ ., data = as.data.frame(X_train_processed))
#       pred <- predict(model, as.data.frame(X_test_processed), type = "class")
#     } else if (model_type == "rf") {
#       model <- randomForest(X_train_processed, y_train)
#       pred <- predict(model, X_test_processed)
#     }
    
#     # Store predictions and calculate metrics
#     predictions[test_indices] <- pred
#     results[[fold]] <- calculate_metrics(pred, y_test)
#   }
  
#   return(list(predictions = predictions, metrics = results))
# }

# # ---- 6. Enhanced Visualization Functions ----
# prepare_metrics_data <- function(results_list) {
#   plot_data <- data.frame()
  
#   for (model in names(results_list)) {
#     metrics <- results_list[[model]]$metrics
    
#     # Extract metrics
#     accuracies <- sapply(metrics, function(x) x$accuracy)
#     sensitivities <- sapply(metrics, function(x) x$sensitivity)
#     specificities <- sapply(metrics, function(x) x$specificity)
    
#     # Combine into data frame
#     model_data <- data.frame(
#       Model = rep(model, 3 * length(accuracies)),
#       Metric = rep(c("Accuracy", "Sensitivity", "Specificity"), each = length(accuracies)),
#       Value = c(accuracies, sensitivities, specificities)
#     )
    
#     plot_data <- rbind(plot_data, model_data)
#   }
  
#   # Convert Model to factor with specific order
#   plot_data$Model <- factor(plot_data$Model, 
#                            levels = c("logistic", "lda", "qda", "naive_bayes", "tree", "rf"),
#                            labels = c("Logistic", "LDA", "QDA", "Naive Bayes", "Decision Tree", "Random Forest"))
  
#   return(plot_data)
# }

# create_metric_plot <- function(data, metric_name) {
#   metric_data <- data[data$Metric == metric_name, ]
  
#   ggplot(metric_data, aes(x = Model, y = Value, fill = Model)) +
#     geom_boxplot(alpha = 0.8) +
#     scale_fill_brewer(palette = "Set2") +
#     theme_minimal() +
#     labs(title = paste(metric_name, "by Model"),
#          y = metric_name,
#          x = "") +
#     theme(
#       plot.title = element_text(size = 12, face = "bold", hjust = 0.5),
#       axis.text.x = element_text(angle = 45, hjust = 1),
#       legend.position = "none",
#       panel.grid.major.x = element_blank(),
#       panel.border = element_rect(fill = NA, color = "gray80"),
#       plot.margin = margin(5, 10, 5, 10)
#     ) +
#     scale_y_continuous(limits = c(0, 1), breaks = seq(0, 1, 0.1))
# }

# plot_classification_results <- function(results_list, title = "Model Performance Comparison") {
#   # Prepare data
#   plot_data <- prepare_metrics_data(results_list)
  
#   # Create individual plots
#   accuracy_plot <- create_metric_plot(plot_data, "Accuracy")
#   sensitivity_plot <- create_metric_plot(plot_data, "Sensitivity")
#   specificity_plot <- create_metric_plot(plot_data, "Specificity")
  
#   # Combine plots
#   combined_plot <- grid.arrange(
#     accuracy_plot, sensitivity_plot, specificity_plot,
#     ncol = 1,
#     top = grid::textGrob(title, gp = grid::gpar(fontsize = 14, fontface = "bold"))
#   )
  
#   return(combined_plot)
# }

# create_summary_table <- function(results_list) {
#   summary_data <- data.frame()
  
#   for (model in names(results_list)) {
#     metrics <- results_list[[model]]$metrics
    
#     # Calculate summary statistics
#     model_summary <- data.frame(
#       Model = model,
#       Metric = c("Accuracy", "Sensitivity", "Specificity"),
#       Mean = c(
#         mean(sapply(metrics, function(x) x$accuracy)),
#         mean(sapply(metrics, function(x) x$sensitivity)),
#         mean(sapply(metrics, function(x) x$specificity))
#       ),
#       SD = c(
#         sd(sapply(metrics, function(x) x$accuracy)),
#         sd(sapply(metrics, function(x) x$sensitivity)),
#         sd(sapply(metrics, function(x) x$specificity))
#       )
#     )
    
#     summary_data <- rbind(summary_data, model_summary)
#   }
  
#   # Create summary table plot
#   summary_plot <- ggplot(summary_data, aes(x = Model, y = Metric)) +
#     geom_tile(aes(fill = Mean), color = "white") +
#     geom_text(aes(label = sprintf("%.3f\n(±%.3f)", Mean, SD)), size = 3) +
#     scale_fill_gradient2(low = "white", high = "#4CAF50", midpoint = 0.5) +
#     theme_minimal() +
#     labs(title = "Summary Statistics",
#          fill = "Mean Value") +
#     theme(
#       axis.text.x = element_text(angle = 45, hjust = 1),
#       plot.title = element_text(size = 12, face = "bold", hjust = 0.5)
#     )
  
#   return(summary_plot)
# }

# # ---- 7. Statistical Comparison Function ----
# perform_mcnemar_tests <- function(results_list) {
#   models <- names(results_list)
#   n_models <- length(models)
#   mcnemar_results <- matrix(NA, n_models, n_models)
#   rownames(mcnemar_results) <- models
#   colnames(mcnemar_results) <- models
  
#   for (i in 1:(n_models-1)) {
#     for (j in (i+1):n_models) {
#       pred_i <- results_list[[models[i]]]$predictions
#       pred_j <- results_list[[models[j]]]$predictions
      
#       # Create contingency table
#       table_ij <- table(pred_i == y, pred_j == y)
      
#       # Perform McNemar's test
#       test_result <- mcnemar.test(table_ij)
#       mcnemar_results[i,j] <- test_result$p.value
#       mcnemar_results[j,i] <- test_result$p.value
#     }
#   }
  
#   diag(mcnemar_results) <- 1
#   return(mcnemar_results)
# }

# # ---- 8. Run Complete Analysis ----
# # Define models to test
# model_types <- c("logistic", "lda", "qda", "naive_bayes", "tree", "rf")

# # Run analysis without preprocessing
# cat("\nRunning analysis without preprocessing...\n")
# results_original <- list()
# for (model in model_types) {
#   cat(sprintf("Training %s model...\n", model))
#   results_original[[model]] <- nested_cv(X, y, model_type = model, 
#                                        preprocess_params = list(scale = FALSE, 
#                                                              remove_correlated = FALSE,
#                                                              handle_outliers = FALSE))
# }

# # Run analysis with preprocessing
# cat("\nRunning analysis with preprocessing...\n")
# results_preprocessed <- list()
# for (model in model_types) {
#   cat(sprintf("Training %s model...\n", model))
#   results_preprocessed[[model]] <- nested_cv(X, y, model_type = model, 
#                                            preprocess_params = list(scale = TRUE, 
#                                                                  remove_correlated = TRUE,
#                                                                  handle_outliers = TRUE))
# }

# # ---- 9. Generate and Save Results ----
# # Create output directory if it doesn't exist
# dir.create("analysis_results", showWarnings = FALSE)

# # Save plots to PDF
# pdf("analysis_results/classification_results.pdf", height = 12, width = 10)

# # Plot results without preprocessing
# plots_original <- plot_classification_results(results_original, 
#     "Model Performance Comparison (Without Preprocessing)")
# summary_original <- create_summary_table(results_original)
# grid.arrange(plots_original, summary_original, ncol = 1, heights = c(3, 1))

# # Plot results with preprocessing
# plots_preprocessed <- plot_classification_results(results_preprocessed, 
#     "Model Performance Comparison (With Preprocessing)")
# summary_preprocessed <- create_summary_table(results_preprocessed)
# grid.arrange(plots_preprocessed, summary_preprocessed, ncol = 1, heights = c(3, 1))

# dev.off()

# # Perform statistical comparisons
# mcnemar_results_original <- perform_mcnemar_tests(results_original)
# mcnemar_results_preprocessed <- perform_mcnemar_tests(results_preprocessed)

# # Save statistical results
# sink("analysis_results/statistical_results.txt")

# cat("\nMcNemar's Test Results (Original Data):\n")
# print(round(mcnemar_results_original, 4))

# cat("\nMcNemar's Test Results (Preprocessed Data):\n")
# print(round(mcnemar_results_preprocessed, 4))

# # Print detailed metrics
# print_model_metrics <- function(results_list, title) {
#   cat("\n", title, "\n", sep="")
#   cat(paste(rep("-", nchar(title)), collapse=""), "\n")
  
#   for (model in names(results_list)) {
#     metrics <- results_list[[model]]$metrics
#     avg_accuracy <- mean(sapply(metrics, function(x) x$accuracy))
#     avg_sensitivity <- mean(sapply(metrics, function(x) x$sensitivity))
#     avg_specificity <- mean(sapply(metrics, function(x) x$specificity))
    
#     sd_accuracy <- sd(sapply(metrics, function(x) x$accuracy))
#     sd_sensitivity <- sd(sapply(metrics, function(x) x$sensitivity))
#     sd_specificity <- sd(sapply(metrics, function(x) x$specificity))
    
#     cat(sprintf("\n%s:\n", model))
#     cat(sprintf("Accuracy: %.4f (±%.4f)\n", avg_accuracy, sd_accuracy))
#     cat(sprintf("Sensitivity: %.4f (±%.4f)\n", avg_sensitivity, sd_sensitivity))
#     cat(sprintf("Specificity: %.4f (±%.4f)\n", avg_specificity, sd_specificity))
#   }
# }

# # Print detailed metrics for both analyses
# cat("\nDetailed Model Metrics:\n")
# cat("=====================\n")
# print_model_metrics(results_original, "Results without Preprocessing")
# print_model_metrics(results_preprocessed, "Results with Preprocessing")

# # Close the output file
# sink()

# # Create comparison plots of preprocessed vs non-preprocessed results
# pdf("analysis_results/preprocessing_comparison.pdf", height = 8, width = 12)

# # Prepare combined data for comparison
# prepare_comparison_data <- function(original_results, preprocessed_results) {
#   original_data <- prepare_metrics_data(original_results)
#   original_data$Preprocessing <- "Without Preprocessing"
  
#   preprocessed_data <- prepare_metrics_data(preprocessed_results)
#   preprocessed_data$Preprocessing <- "With Preprocessing"
  
#   rbind(original_data, preprocessed_data)
# }

# comparison_data <- prepare_comparison_data(results_original, results_preprocessed)

# # Create comparison plots
# for (metric in c("Accuracy", "Sensitivity", "Specificity")) {
#   metric_data <- comparison_data[comparison_data$Metric == metric, ]
  
#   p <- ggplot(metric_data, aes(x = Model, y = Value, fill = Preprocessing)) +
#     geom_boxplot(position = position_dodge(width = 0.8), alpha = 0.8) +
#     scale_fill_brewer(palette = "Set2") +
#     theme_minimal() +
#     labs(title = paste(metric, "Comparison: Preprocessing Effect"),
#          y = metric,
#          x = "") +
#     theme(
#       plot.title = element_text(size = 12, face = "bold", hjust = 0.5),
#       axis.text.x = element_text(angle = 45, hjust = 1),
#       legend.position = "top",
#       panel.grid.major.x = element_blank(),
#       panel.border = element_rect(fill = NA, color = "gray80"),
#       plot.margin = margin(5, 10, 5, 10)
#     ) +
#     scale_y_continuous(limits = c(0, 1), breaks = seq(0, 1, 0.1))
  
#   print(p)
# }

# dev.off()

# # Print completion message
# cat("\nAnalysis complete! Results have been saved to the 'analysis_results' directory.\n")
# cat("Generated files:\n")
# cat("1. classification_results.pdf - Contains performance plots and summary tables\n")
# cat("2. preprocessing_comparison.pdf - Shows the effect of preprocessing\n")
# cat("3. statistical_results.txt - Contains detailed metrics and statistical tests\n")


Running analysis without preprocessing...
Training logistic model...


ERROR: Error in preprocess(X_train_processed): impossible de trouver la fonction "preprocess"
